<a href="https://colab.research.google.com/github/Cansu-byte/PythonProject/blob/main/MR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from albumentations import (Compose, HorizontalFlip, Rotate, ShiftScaleRotate, RandomBrightnessContrast, HueSaturationValue)
from albumentations.core.composition import OneOf
import os
import numpy as np
import pandas as pd




In [8]:


def load_data_with_augmentation(data_dir, augment=False):
    images = []
    labels = []
    classes = ['Glioma', 'Meningioma', 'No Tumor', 'Pituitary']
    for  class_id, class_name in enumerate(classes):
        img_dir = os.path.join(data_dir, class_name, 'images')#görüntü yollari belirtilir.
        label_dir = os.path.join(data_dir, class_name, 'labels')

      # Check if the directories exist
        if not os.path.exists(img_dir):
            print(f"Warning: Directory not found: {img_dir}")
            continue
        if not os.path.exists(label_dir):
            print(f"Warning: Directory not found: {label_dir}")
            continue

        for img_name in os.listdir(img_dir):                  #os.listdir(img_dir) kullanarak img_dir içindeki tüm dosyaların isimlerini alırız.
            img_path = os.path.join(img_dir, img_name)        #Dosya yollari olusturulur.
            label_path = os.path.join(label_dir, img_name.replace('.jpg', '.txt'))


            img = cv2.imread(img_path)                       #Görüntüyü okuma, boyutlandirma, geçersizleri atlama
            if img is None:
                continue
            img = cv2.resize(img, (224, 224))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            if augment:                                       # Veri artirma islemi ile modelin veri varyasyonlarina( yatay çevirme, döndürme, kaydırma, ölçekleme vb.) karsi dayanilikligi arttirir.
                aug = Compose([
                    HorizontalFlip(),
                    OneOf([
                        Rotate(limit=30),
                        ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=30)
                    ], p=0.5),
                    OneOf([
                        RandomBrightnessContrast(),
                        HueSaturationValue()
                    ], p=0.5)
                ])
                img = aug(image=img)['image']


            if os.path.exists(label_path):                    #Her bir görüntü dosyası için ilgili etiket dosyasını kontrol eder ve etiketi okur. Etiketler geçerli ve okunabilir olduğunda, görüntüyü ve etiketleri listelere ekler.
                with open(label_path, 'r') as f:
                    label = f.readline().strip().split()
                    if len(label) > 0:
                        class_id = int(label[0])
                    else:
                        continue
            else:
                 continue

            images.append(img)
            labels.append(class_id)

    images = np.array(images)
    labels = np.array(labels)
    return images, labels

train_dir = os.path.join( 'Train')
val_dir = os.path.join( 'Val')

train_images, train_labels = load_data_with_augmentation(train_dir, augment=True)
val_images, val_labels = load_data_with_augmentation(val_dir, augment=False)

train_images.shape, val_images.shape


((0,), (0,))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [13]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(train_images, train_labels, batch_size=32)  # Bu üreticiler, modelin eğitim ve doğrulama süreçlerinde kullanılacak veri akışını yönetir.
val_generator = val_datagen.flow(val_images, val_labels, batch_size=32)

((4737, 224, 224, 3), (509, 224, 224, 3))

In [1]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))  #Modeli yükleme

for layer in base_model.layers:                                                           #Katmanları doldurma
    layer.trainable = False

x = base_model.output                                                                     #Özelleştirilmiş katmanlar
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(4, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)                                #Model Tanımlama



NameError: name 'ResNet50' is not defined